# Ramp of the simulation

In [3]:
#-----Importa paquetes escenciales

from __future__ import division
import hoomd
import hoomd.md

#-----Define las variables importantes

p_prev = 0.5
p_new = 0.8
t_max = 5.0 
time_steps = 2.0e5; 

#-----Crea un contexto de simulación

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Extrae la configuración del centro de masa de las partículas

system = hoomd.init.read_gsd("T_5.0_P_0.5_initializing.gsd",frame = -1)

#-----Define cada mesógeno en el marco de referencia local de cada centro de masa.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Declara que las moléculas creadas son cuerpos rígidos

rigid.create_bodies();

#-----Define la energía potencial

nl = hoomd.md.nlist.tree()

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist=nl)
lj.set_params(mode='shift')

#-----Define la interacción entre las especies de la simulación

lj.pair_coeff.set('M','M', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('A','A', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('M','A', epsilon = 1.0, sigma = 1.0)

#-----Selecciona un integrador standard

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define el grupo sobre el cual se integraran las ecuaciones de movimiento

mesogens = hoomd.group.rigid_center();

#----Integra usando un NPT

pressure = hoomd.variant.linear_interp(points = [(0,p_prev), (time_steps, p_new)])

npt = hoomd.md.integrate.npt(group = mesogens, kT = t_max , tau = 1.5, tauP = 1.5, P = pressure);

#-----Guarda la información de la simulación en un archivo .log y en un .gsd

log_file = "T_" + str(t_max) + "_P_" + str(p_new) + "_ramp.log"
gsd_file = "T_" + str(t_max) + "_P_" + str(p_new) + "_ramp.gsd" 
meso_gsd_file = "T_CM_" + str(t_max) + "_P_" + str(p_new) + "_ramp.gsd"
log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite=True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite=True); 

notice(2): Group "all" created containing 9000 particles
notice(2): constrain.rigid(): Removing all particles part of rigid bodies (except central particles).Particle tags may change.
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for molecular simulation
  of colloidal systems using graphics processing units", Computer Physics
  Communications 203 (2016) 45--52
-----
notice(2): Group "rigid_center" created containing 1000 particles


In [ ]:
hoomd.run(time_steps)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9000
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 335243 / 535000 | TPS 24.1605 | ETA 02:17:47
Time 00:00:20 | Step 335500 / 535000 | TPS 25.5705 | ETA 02:10:01
Time 00:00:30 | Step 335758 / 535000 | TPS 25.7514 | ETA 02:08:57
Time 00:00:40 | Step 336034 / 535000 | TPS 27.5892 | ETA 02:00:11
Time 00:00:50 | Step 336305 / 535000 | TPS 27.0937 | ETA 02:02:13
Time 00:01:00 | Step 336582 / 535000 | TPS 27.5594 | ETA 01:59:59
Time 00:01:10 | Step 336865 / 535000 | TPS 28.2846 | ETA 01:56:45
Time 00:01:20 | Step 337131 / 535000 | TPS 26.534 | ETA 02:04:17
Time 00:01:30 | Step 337395 / 535000 | TPS 26.3007 | ETA 02:05:13
Time 00:01:40 | Step 337670 / 535000 | TPS 27.4571 | ETA 01:59:46
Time 00:01:50 | Step 337935 / 535000 | TPS 26.4712 | ETA 02:04:04
Time 00:02:00 | Step 338206 / 53